Lambda School Data Science

*Unit 2, Sprint 3, Module 2*

---


# Wrangle ML datasets 🍌


In today's lesson, we’ll work with a dataset of [3 Million Instacart Orders, Open Sourced](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2)!



### Setup

In [1]:
# Download data
import requests

def download(url):
    filename = url.split('/')[-1]
    print(f'Downloading {url}')
    r = requests.get(url)
    with open(filename, 'wb') as f:
        f.write(r.content)
    print(f'Downloaded {filename}')

download('https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz')

Downloaded instacart_online_grocery_shopping_2017_05_01.tar.gz


In [2]:
# Uncompress data
import tarfile
tarfile.open('instacart_online_grocery_shopping_2017_05_01.tar.gz').extractall()

In [3]:
# Change directory to where the data was uncompressed
%cd instacart_2017_05_01

C:\Users\jonma\Programming\DS-Unit-2-Applied-Modeling\module2-wrangle-ml-datasets\instacart_2017_05_01


In [4]:
# Print the csv filenames
from glob import glob
for filename in glob('*.csv'):
    print(filename)

aisles.csv
departments.csv
orders.csv
order_products__prior.csv
order_products__train.csv
products.csv


### For each csv file, look at its shape & head 

In [12]:
import pandas as pd
# to format the head. HTMLE
from IPython.display import display

for filename in glob('*.csv'):
    df = pd.read_csv(filename)
    print('\n', filename, df.shape)
    display(df.head())


 aisles.csv (134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation



 departments.csv (21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol



 orders.csv (3421083, 7)


...

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0



 order_products__train.csv (1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1



 products.csv (49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


## The original task was complex ...

[The Kaggle competition said,](https://www.kaggle.com/c/instacart-market-basket-analysis/data):

> The dataset for this competition is a relational set of files describing customers' orders over time. The goal of the competition is to predict which products will be in a user's next order.

> orders.csv: This file tells to which set (prior, train, test) an order belongs. You are predicting reordered items only for the test set orders.

Each row in the submission is an order_id from the test set, followed by product_id(s) predicted to be reordered.

> sample_submission.csv: 
```
order_id,products
17,39276 29259
34,39276 29259
137,39276 29259
182,39276 29259
257,39276 29259
```

## ... but we can simplify!

Simplify the question, from "Which products will be reordered?" (Multi-class, [multi-label](https://en.wikipedia.org/wiki/Multi-label_classification) classification) to **"Will customers reorder this one product?"** (Binary classification)

Which product? How about **the most frequently ordered product?**

# Questions:

- What is the most frequently ordered product?
- How often is this product included in a customer's next order?
- Which customers have ordered this product before?
- How can we get a subset of data, just for these customers?
- What features can we engineer? We want to predict, will these customers reorder this product on their next order?

## What was the most frequently ordered product?

In [13]:
order_products__train = pd.read_csv('order_products__train.csv')
order_products__train['product_id'].value_counts()

24852    18726
13176    15480
21137    10894
21903     9784
47626     8135
         ...  
44256        1
2764         1
4815         1
43736        1
46835        1
Name: product_id, Length: 39123, dtype: int64

In [18]:
products = pd.read_csv('products.csv')
products.loc[24851, 'product_name']
# df.loc[row,column], this doesn't search any values, just gives the value in that location.

'Banana'

In [17]:
combined = pd.merge(order_products__train, products, on='product_id', how='inner')
combined['product_name'].value_counts()

Banana                                                                        18726
Bag of Organic Bananas                                                        15480
Organic Strawberries                                                          10894
Organic Baby Spinach                                                           9784
Large Lemon                                                                    8135
                                                                              ...  
Active Life Fragrance Free Deodorant Stick                                        1
Tomato Juice From Concentrate                                                     1
Seriously Sharp Premium Naturally Aged Cheddar Cheese                             1
Regular Strength 500 Peppermint Chewable Tablets Antacid Calcium Carbonate        1
Sazon Seasoning                                                                   1
Name: product_name, Length: 39123, dtype: int64

## How often is this product included in a customer's next order?

There are [three sets of data](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b):

> "prior": orders prior to that users most recent order (3.2m orders)  
"train": training data supplied to participants (131k orders)  
"test": test data reserved for machine learning competitions (75k orders)

Customers' next orders are in the "train" and "test" sets. (The "prior" set has the orders prior to the most recent orders.)

We can't use the "test" set here, because we don't have its labels (only Kaggle & Instacart have them), so we don't know what products were bought in the "test" set orders.

So, we'll use the "train" set. It currently has one row per product_id and multiple rows per order_id.

But we don't want that. Instead we want one row per order_id, with a binary column: "Did the order include the product?"

Let's wrangle!

In [20]:
train = combined
train

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id
0,1,49302,1,1,Bulgarian Yogurt,120,16
1,816049,49302,7,1,Bulgarian Yogurt,120,16
2,1242203,49302,1,1,Bulgarian Yogurt,120,16
3,1383349,49302,11,1,Bulgarian Yogurt,120,16
4,1787378,49302,8,0,Bulgarian Yogurt,120,16
...,...,...,...,...,...,...,...
1384612,3420011,1528,12,0,Sprinkles Decors,97,13
1384613,3420084,47935,20,0,Classic Original Lip Balm SPF 12,73,11
1384614,3420084,9491,21,0,Goats Milk & Chai Soap,25,11
1384615,3420088,16380,12,0,Stevia Sweetener,97,13


In [22]:
train['Banana'] = train['product_name']== 'Banana'


In [31]:
train['Banana'].value_counts(normalize=True)

False    0.986476
True     0.013524
Name: Banana, dtype: float64

In [26]:
#for each order, were any of the products Bananas?

train_wrangled = train.groupby('order_id')['Banana'].any().reset_index()

In [30]:
train_wrangled['Banana'].value_counts(normalize=True)

False    0.857281
True     0.142719
Name: Banana, dtype: float64

## Which customers have ordered this product before?

- Customers are identified by `user_id`
- Products are identified by `product_id`

Do we have a table with both these id's? (If not, how can we combine this information?)

In [38]:
# Order_id for the orders that included bananas
train[train.Banana==True]

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,Banana
129688,226,24852,2,0,Banana,24,4,True
129689,473,24852,2,0,Banana,24,4,True
129690,878,24852,2,1,Banana,24,4,True
129691,1042,24852,1,1,Banana,24,4,True
129692,1139,24852,1,1,Banana,24,4,True
...,...,...,...,...,...,...,...,...
148409,3419531,24852,2,1,Banana,24,4,True
148410,3419542,24852,6,0,Banana,24,4,True
148411,3419629,24852,5,1,Banana,24,4,True
148412,3420088,24852,9,1,Banana,24,4,True


In [39]:
# find customers associated with those order_ids
banana_order_ids = train[train['Banana']==True].order_id

In [40]:
# Look at orders table 
orders = pd.read_csv('orders.csv')
orders.sample(n=5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
620131,1068755,37396,prior,16,2,14,6.0
1605090,1002964,96370,prior,4,4,10,16.0
593535,858001,35767,prior,13,6,16,30.0
724316,460384,43592,prior,33,3,14,9.0
1789069,356500,107459,prior,2,3,14,14.0


In [54]:
# for each order id, is it in a certain list

banana_orders  orders[orders['order_id'].isin(banana_order_ids)]

SyntaxError: invalid syntax (<ipython-input-54-0fbfc731db17>, line 3)

In [46]:
banana_orders = orders[orders['order_id'].isin(banana_order_ids)]

In [47]:
# in the orders table, which users have bought bananas?
banana_orders.user_id.unique()

array([     2,     34,     43, ..., 206196, 206205, 206209], dtype=int64)

## How can we get a subset of data, just for these customers?

We want *all* the orders from customers who have *ever* bought the product.

(And *none* of the orders from customers who have *never* bought the product.)

In [50]:
banana_users = banana_orders.user_id.unique()

In [52]:
orders[orders['user_id'].isin(banana_users)]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
11,2168274,2,prior,1,2,11,NaN
12,1501582,2,prior,2,5,10,10.0
13,1901567,2,prior,3,1,10,3.0
14,738281,2,prior,4,2,10,8.0
15,1673511,2,prior,5,3,11,8.0
...,...,...,...,...,...,...,...
3421078,2266710,206209,prior,10,5,18,29.0
3421079,1854736,206209,prior,11,4,10,30.0
3421080,626363,206209,prior,12,1,12,18.0
3421081,2977660,206209,prior,13,1,12,7.0


## What features can we engineer? We want to predict, will these customers reorder this product on their next order?
- products per order
- time of day
- have they reordered bananas before
- other fruit they buy
- size of orders (smaller orders on avg are less likely to be reordering any particular product)

Frequency
    - % of orders
    -time between banana orders
    - raw count: total orders
    
Recency of banana orders
    -n days since you ordered (perhaps non-monotonic)

In [58]:
orders[orders['user_id']==2]

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
11,2168274,2,prior,1,2,11,NaN
12,1501582,2,prior,2,5,10,10.0
13,1901567,2,prior,3,1,10,3.0
14,738281,2,prior,4,2,10,8.0
15,1673511,2,prior,5,3,11,8.0
16,1199898,2,prior,6,2,9,13.0
17,3194192,2,prior,7,2,12,14.0
18,788338,2,prior,8,1,15,27.0
19,1718559,2,prior,9,2,9,8.0
20,1447487,2,prior,10,1,11,6.0
